### Imports and setup

In [ ]:
import pandas as pd
import yaml
import matplotlib.pyplot as plt
import warnings
import scienceplots
from statsmodels.tsa.api import VAR

import sys
import os
from dotenv import load_dotenv

load_dotenv()
REPO_PATH = os.getenv("REPO_PATH")

# Import main utility functions
sys.path.insert(0, rf'{REPO_PATH}src')
from utils.var_utils import plot_criterion, forecast_var, save_var_info
from utils.main_utils import load_processed, load_variables

plt.style.use('science')
warnings.filterwarnings("ignore")

### Load data

In [ ]:

var_config = load_variables()
SELECTED_FEATURES = [
    *var_config['BASE_VAR'],
    *var_config['TEMP_VAR']
]

FUTURES = ['CLc1', 'LCOc1']
IDENTIFIER = 'B'

dfs = load_processed(FUTURES)

### Find optimal lags

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(7, 3), dpi=200)

for i, future in enumerate(FUTURES):
    model = VAR(dfs[future][SELECTED_FEATURES])
    lag_order = model.select_order(100, trend='c')
    plot_criterion(lag_order, axs[i], FUTURES[i])
    axs[i].set_ylabel('Score')

fig.tight_layout()
fig.savefig(f'images/OLO_{IDENTIFIER}.png')

### Fit Model

In [ ]:
VAR_PARAMS = {
    'features': SELECTED_FEATURES,
    'target': 'REALIZED_VOL',
    'lags': 11,
    'test_size': 0.2
}

FUTURE = 'LCOc1'

results = forecast_var(FUTURE, VAR_PARAMS)


current_dt: str = pd.Timestamp.now().strftime('%Y.%m.%d_%H.%M')
model_name: str = f'{FUTURE}_VAR_{IDENTIFIER}_{current_dt}'

save_var_info(model_name, VAR_PARAMS)